## Plot supervision signal example

In [ ]:
from wild_visual_navigation import WVN_ROOT_DIR
from wild_visual_navigation.supervision_generator import SupervisionGenerator
from wild_visual_navigation.learning.dataset import TwistDataset
import wild_visual_navigation.visu.paper_colors as pc
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from wild_visual_navigation import WVN_ROOT_DIR


%matplotlib inline

In [ ]:
# Prepare dataset
root = str(os.path.join(WVN_ROOT_DIR, "assets/twist_measurements"))
current_filename = "current_robot_twist.csv"
desired_filename = "desired_robot_twist.csv"
data = TwistDataset(root, current_filename, desired_filename, seq_size=1, smoothing_window=100)

# Prepare traversability generator
sg = SupervisionGenerator(
    device="cpu",
    kf_process_cov=0.5,
    kf_meas_cov=1,
    kf_outlier_rejection="none",
    kf_outlier_rejection_delta=0.5,
    sigmoid_slope=20,
    sigmoid_cutoff=0.2,  # 0.2
    untraversable_thr=0.1,
    time_horizon=0.05,
)

In [ ]:
# Saved data list
saved_data = []

# Iterate the dataset
for i in range(data.size):
    # Get samples
    t, curr, des = data[i]

    # Update traversability
    trav, trav_cov, is_untrav = sg.update_velocity_tracking(curr[0], des[0], max_velocity=0.7, velocities=["vx", "vy"])
    saved_data.append([t.item(), curr.norm().item(), des.norm().item(), trav.item(), trav_cov.item(), is_untrav])

df = pd.DataFrame(saved_data, columns=["ts", "curr", "des", "trav", "trav_cov", "is_untraversable"])

# Normalize by time
df["ts"] = df["ts"] - df["ts"].values[0]
df["trav_upper"] = df["trav"] + df["trav_cov"]
df["trav_lower"] = df["trav"] - df["trav_cov"]

In [ ]:
# From https://matplotlib.org/stable/gallery/lines_bars_and_markers/gradient_bar.html


def gradient_image(ax, extent, direction=0.3, cmap_range=(0, 1), **kwargs):
    """
    Draw a gradient image based on a colormap.

    Parameters
    ----------
    ax : Axes
        The axes to draw on.
    extent
        The extent of the image as (xmin, xmax, ymin, ymax).
        By default, this is in Axes coordinates but may be
        changed using the *transform* keyword argument.
    direction : float
        The direction of the gradient. This is a number in
        range 0 (=vertical) to 1 (=horizontal).
    cmap_range : float, float
        The fraction (cmin, cmax) of the colormap that should be
        used for the gradient, where the complete colormap is (0, 1).
    **kwargs
        Other parameters are passed on to `.Axes.imshow()`.
        In particular useful is *cmap*.
    """
    phi = direction * np.pi / 2
    v = np.array([np.cos(phi), np.sin(phi)])
    X = np.array([[v @ [1, 0], v @ [1, 1]], [v @ [0, 0], v @ [0, 1]]])
    a, b = cmap_range
    X = a + (b - a) / X.max() * X

    X = np.tile(np.linspace(1, 0, 250), (1, 1)).T
    im = ax.imshow(X, extent=extent, interpolation="bilinear", vmin=0, vmax=1, **kwargs)
    return im

In [ ]:
t1 = 94
t2 = 113.5
df2 = df[df["ts"] > t1]
df2 = df2[df2["ts"] < t2]
df2["ts"] = df2["ts"] - df2["ts"].values[0]

width_half_inch, width_double_inch = 88.9 / 25.4, 182.0 / 25.4
height_inch = 30 / 25.4
scale = 1.5

N_plots = 2

fig, axs = plt.subplots(
    N_plots,
    constrained_layout=True,
    sharex=True,
    figsize=(width_half_inch * scale, N_plots * height_inch * scale),
    dpi=300,
)

# Top plot
axs[0].plot(df2["ts"], df2["curr"], label="Current", color=pc.paper_colors_rgba_f["blue"], linewidth=2)
axs[0].plot(df2["ts"], df2["des"], label="Reference", color="k", linewidth=2)
axs[0].set_ylabel("Velocity [m/s]")
axs[0].set_title("Velocities", y=1.0, pad=-14)
axs[0].legend(loc="upper right")
axs[0].set_ylim(0.0, 1.5)


# Bottom plot
upper_limit = 1.3

# Plot line
ax = axs[1]
ax.plot(df2["ts"], df2["trav"], label="Traversability", color="k", linewidth=0.5)

# Fake the gradient by painting the background with the gradient and then overlaying white fill on top
# Extent is (left, right, bottop, top) limits
extent = (0, 1, 0, 1.0 / 1.3)

cmap = sns.color_palette("RdYlBu", as_cmap=True)

gradient_image(
    ax,
    extent=extent,
    direction=0,
    transform=ax.transAxes,
    cmap_range=(0, 1),
    cmap=cmap,
)
ax.fill_between(
    df2["ts"], df2["trav"], np.ones(df2["trav"].shape) * upper_limit, color="white", alpha=1.0, linewidth=0.0
)


ax.set_ylabel("Score")
ax.set_title(r"Traversability $\tau$", y=1.0, pad=-14)
# axs[1].legend(loc="upper right")
ax.set_ylim(0.0, upper_limit)
ax.set_xlim(df2["ts"].values[0], df2["ts"].values[-1])
ax.set_aspect("auto")

plt.xlabel("Time [s]")
# plt.tight_layout()

fig.savefig(Path(os.path.join(WVN_ROOT_DIR), "results/paper/traversability_score.pdf"))